In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,multilabel_confusion_matrix

In [2]:
df = pd.read_excel('1.xlsx')
df

,i,post,prefLabel,compulsion,obsession,yy
0,1,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination..."
1,2,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
2,3,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination..."
3,4,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination..."
4,5,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination..."
...,...,...,...,...,...,...
1556,1557,"Hi all,\nI think I know the answers before I e...","['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
1557,1558,"Hi folks,\nI'm really struggling at the mo. I'...","['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination..."
1558,1559,Well hello everyone\nAs you can see one of my ...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination..."
1559,1560,Well hello!\n\nIve been struggling with ocd in...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination..."


In [4]:
df['post'][0]

'Hi -\nSo I haven\'t been on here since December since I was recovering. I\'ve been on prozac since I was 18 (now 24) for depression but I was diagnosed with OCD in January. My psych bumped me up to a higher dosage of prozac - 80mg - which was doing a great job. But I wanted to go off of it because it was making me gain weight and now I\'m freaking out!! My OCD revolves around cheating like some of you know...and my most recent """"obsession"""" (I put quotes around it cuz i question whether its an obsession or if its real) is that... well let me start from the beginning. I went to my fiancé\'s friends place and we all had a few drinks and played card games and such. I was with my fiance the whole time except for when I went to the bathroom. I remember everything from the night despite the alcohol in my system. We came home and continued drinking (just me and my fiance) and I eventually blacked out. I ended up having a dream about cheating on my partner with someone that I had seen ear

In [5]:
df['yy'].describe()

count                                                  1561
unique                                                  962
top       Intrusive thoughts, Aggressive intrusive thoug...
freq                                                     95
Name: yy, dtype: object

In [6]:
most_frequent_term = df['yy'].value_counts().idxmax()
frequency_of_most_frequent_term = df['yy'].value_counts().max()

print("Most Frequent Term:", most_frequent_term)
print("Frequency of Most Frequent Term:", frequency_of_most_frequent_term)

Most Frequent Term: Intrusive thoughts, Aggressive intrusive thoughts, Contamination intrusive thoughts, Doubt intrusive thoughts, Hoarding intrusive thoughts, Religious intrusive thoughts, Sexual intrusive thoughts, Somatic intrusive thoughts, Symmetry intrusive thoughts, Thought, morbid thoughts, Mental image, Homicidal thoughts, Persistent thoughts, Irrational thoughts, Suicidal ideation, Sexual thought, Aggressive thought, Intrusive image, Contamination thought, Doubt thought, Hoarding thought, Obsessional thoughts, Religious thought, Somatic thought, Symmetry thought, Aggressive Intrusive image, Contamination intrusive image, Doubt intrusive image, Hoardign intrusive image, Religious intrusive image, Sexual intrusive image, Somatic intrusive image, Symmetry intrusive image
Frequency of Most Frequent Term: 95


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bkj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bkj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bkj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [14]:
process_df = df.copy()
process_df

,i,post,prefLabel,compulsion,obsession,yy
0,1,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination..."
1,2,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
2,3,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination..."
3,4,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination..."
4,5,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination..."
...,...,...,...,...,...,...
1556,1557,"Hi all,\nI think I know the answers before I e...","['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
1557,1558,"Hi folks,\nI'm really struggling at the mo. I'...","['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination..."
1558,1559,Well hello everyone\nAs you can see one of my ...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination..."
1559,1560,Well hello!\n\nIve been struggling with ocd in...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination..."


In [15]:
#apply the process of cleaning for the train and test data
process_df = clean_text(process_df)
process_df['post'] = process_df['post'].apply(process_text)
process_df['yy'] = process_df['yy'].apply(process_text)
process_df['clean_text'] = process_df['clean_text'].apply(process_text)
process_df.head()


,i,post,prefLabel,compulsion,obsession,yy,clean_text
0,1,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1,2,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,compulsion intrusive thought aggressive intrus...,compulsion intrusive thought aggressive intrus...
2,3,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
3,4,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
4,5,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...


In [11]:
process_df['post'][0]

'hi since december since recovering prozac since 18 24 depression diagnosed ocd january psych bumped higher dosage prozac 80mg great job wanted go making gain weight freaking ocd revolves around cheating like know recent obsession put quote around cuz question whether obsession real well let start beginning went fiancé friend place drink played card game fiance whole time except went bathroom remember everything night despite alcohol system came home continued drinking fiance eventually blacked ended dream cheating partner someone seen earlier night well really cheating taking guy hand placing odd place image head nothing else really remember waking realize dream think stirred went right back sleep even remember keep thinking dream actually happened fiance keep telling whole time nothing happened think day driving nut always question whether really ocd doc diagnosed make feel better everything true real appt psych tomorrow think going write script zoloft apparently ssri weight gain ssr

#### SPLIT CLEANED DATE TO TRAIN,TEST,VAL DATASETS

In [136]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(process_df, test_size=test_split, stratify=process_df[['compulsion', 'obsession']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['compulsion', 'obsession']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 1248
Number of rows in validation set: 157
Number of rows in test set: 156


In [13]:
process_df.groupby(['compulsion','obsession']).size()

compulsion  obsession
0           1            943
1           0            370
            1            248
dtype: int64

### CASE 1: USING POST(X) AND COMPULSION,OBSESSION(Y) for training

In [137]:
train_df

,i,post,prefLabel,compulsion,obsession,yy,clean_text
681,682,morning everyone work afternoon thought would ...,"['Anxiety disorder', 'obsession', 'Thought', '...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
796,797,know anyone noticed past week posting negative...,"['OCD', 'exhaustion', 'talking', 'Thought', 'o...",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1086,1087,afford time cbt therapy right trying unsuccess...,"['Intrusive thoughts', 'Thought', 'Food', 'vom...",0,1,intrusive thought aggressive intrusive thought...,intrusive thought aggressive intrusive thought...
166,167,anyone found medication helped stop compulsion...,"['Medication', 'Rituals', 'compulsion']",1,0,compulsion compulsion medication compulsive be...,compulsion compulsion medication compulsive be...
498,499,went girl work last night nightclub jus finish...,"['toilet', 'dancing', 'OCD', 'obsession', 'Tho...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
...,...,...,...,...,...,...,...
11,12,hi hope everyone okay posted honest really bot...,"['Intrusive thoughts', 'Thought', 'OCD', 'depr...",0,1,intrusive thought aggressive intrusive thought...,intrusive thought aggressive intrusive thought...
684,685,hi everyone wondered anyone could help high an...,"['Intrusive thoughts', 'Thought', 'OCD', 'anxi...",0,1,intrusive thought aggressive intrusive thought...,intrusive thought aggressive intrusive thought...
875,876,ocd obsession everywhere morning que panic att...,"['OCD', 'obsession', 'anger']",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1509,1510,hi started cbt area going well one two checkin...,"['checking', 'Rituals', 'Intrusive thoughts', ...",0,1,intrusive thought aggressive intrusive thought...,intrusive thought aggressive intrusive thought...


In [35]:
train_x,train_y = train_df['post'], train_df[['compulsion','obsession']]
test_x,test_y = test_df['post'], test_df[['compulsion','obsession']]
val_x,val_y = val_df['post'], val_df[['compulsion','obsession']]

In [15]:
train_x,train_y

(1258    hi guy wanted put post see anyone relate time ...
 466     hey everyone quickie got ipod touch pretty app...
 825     posted earlier today asking mod opinion compul...
 849     hi guy trying identify whereabouts mind moment...
 949     really want post seems people parallel would r...
                               ...                        
 469     recently found ocd well sure decided find mess...
 53      first bout ocd started age 7 terrified aid sub...
 429     hi currently struggling unwanted intrusive tho...
 275     condition may explain always personal opinion ...
 415     go detail big obsession get car car namely aud...
 Name: post, Length: 1248, dtype: object,
       compulsion  obsession
 1258           1          0
 466            1          0
 825            1          0
 849            1          0
 949            0          1
 ...          ...        ...
 469            0          1
 53             0          1
 429            0          1
 275            0  

#### USING HUGGING FACE

###### MODEL TO PREDICT COMPULSION

In [44]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y['compulsion'], dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y['compulsion'], dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y['compulsion'], dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )

history = model.fit(
    train_encodings,
    train_labels,
    validation_data=(test_encodings, test_labels),
    epochs=3,
    batch_size=32
)

# Evaluate the model
predictions = model.predict(test_encodings)
predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
39/39 [==============================] - 1749s 44s/step - loss: 0.6710 - accuracy: 0.6050 - val_loss: 0.6316 - val_accuracy: 0.6090
Epoch 2/3
39/39 [==============================] - 1692s 44s/step - loss: 0.4197 - accuracy: 0.8373 - val_loss: 0.3335 - val_accuracy: 0.9038
Epoch 3/3
5/5 [==============================] - 51s 9s/step
Accuracy: 0.8974358974358975
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.99      0.92        94
           1       0.98      0.76      0.85        62

    accuracy                           0.90       156
   macro avg       0.92      0.87      0.89       156
weighted avg       0.91      0.90      0.89       156



In [46]:
model.save_pretrained('hugging_face/compulsion_model')

In [50]:
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
val_encodings

{'input_ids': array([[  101,  2514,  5905, ...,     0,     0,     0],
        [  101, 10089, 17418, ...,     0,     0,     0],
        [  101,  7632,  2047, ...,  2518,  2156,   102],
        ...,
        [  101,  7632,  2767, ...,     0,     0,     0],
        [  101,  7632,  2146, ...,     0,     0,     0],
        [  101,  7632,  3124, ...,     0,     0,     0]]),
 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [64]:
val_prediction =  model.predict(val_encodings)
pred = tf.argmax(val_prediction.logits,axis=1)
pred

5/5 [==============================] - 45s 8s/step


<tf.Tensor: shape=(157,), dtype=int64, numpy=
array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0], dtype=int64)>

In [61]:
new_val_df = val_df[['post','compulsion']]
new_val_df['prediction'] = pred.numpy()
new_val_df

C:\Users\bkj\AppData\Local\Temp\ipykernel_22684\3645292725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_val_df['prediction'] = pred.numpy()


,post,compulsion,prediction
412,feel guilty posting rationally know girlfriend...,0,0
843,anxiety obsessional thought wondered anyone co...,1,1
147,hi new 16 think might ocd intrusive thought ce...,1,0
62,posted 3 week 20mg fluoxetine medication must ...,1,1
712,hiya fairly new board hope many people mind as...,0,0
...,...,...,...
1201,well coming day avoiding compulsion wanted ask...,1,1
587,got angry mum earlier tried restrain next knif...,0,0
1044,hi friend bdd realized similar ocd think seems...,1,1
944,hi long time since last posted thing generally...,0,0


In [65]:
loaded_compulsion_model = TFAutoModelForSequenceClassification.from_pretrained('hugging_face/compulsion_model')
loaded_compulsion_model

Some layers from the model checkpoint at hugging_face/compulsion_model were not used when initializing TFBertForSequenceClassification: ['dropout_227']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at hugging_face/compulsion_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [66]:
loaded_val_prediction =  loaded_compulsion_model.predict(val_encodings)
loaded_pred = tf.argmax(loaded_val_prediction.logits,axis=1)
loaded_pred

5/5 [==============================] - 58s 9s/step


<tf.Tensor: shape=(157,), dtype=int64, numpy=
array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0], dtype=int64)>

###### MODEL TO PREDICTION ON OBSESSION

In [68]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_obsession = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y['obsession'], dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y['obsession'], dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y['obsession'], dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
model_obsession.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model_obsession.fit(
    train_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=3,
    batch_size=32
)

# Evaluate the model
predictions = model.predict(test_encodings)
predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
31/39 [======================>.......] - ETA: 7:06 - loss: 0.5471 - accuracy: 0.7631

KeyboardInterrupt: 

##### MODEL FOR ALL LABELS

In [73]:

# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)


history = model.fit(
    train_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=3,
    batch_size=32
)
predictions = model.predict(test_encodings)
predicted_labels = (tf.sigmoid(predictions.logits) > 0.5).numpy()  # Threshold for multi-label classification

# Evaluation
accuracy = accuracy_score(test_y, predicted_labels)
confusion_matrix = multilabel_confusion_matrix(test_y, predicted_labels)
classification_rep = classification_report(test_y, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", confusion_matrix)
print("Classification Report:\n", classification_rep)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
39/39 [==============================] - 2029s 50s/step - loss: 0.6152 - accuracy: 0.6042 - val_loss: 0.5831 - val_accuracy: 0.6051
Epoch 2/3
39/39 [==============================] - 1654s 42s/step - loss: 0.4556 - accuracy: 0.7957 - val_loss: 0.3164 - val_accuracy: 0.9363
Epoch 3/3
5/5 [==============================] - 89s 15s/step
Accuracy: 0.7948717948717948
Confusion Matrix:
 [[[ 94   0]
  [ 15  47]]

 [[ 30   7]
  [ 17 102]]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.76      0.86        62
           1       0.94      0.86      0.89       119

   micro avg       0.96      0.82      0.88       181
   macro avg       0.97      0.81      0.88       181
weighted avg       0.96      0.82      0.88       181
 samples avg       0.96      0.88      0.90       181



In [74]:
model.save_pretrained('hugging_face/all_model')

In [80]:
predicted_labels = predicted_labels.astype('int')
predicted_labels

array([[0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0,

In [83]:
new_test_df = test_df[['post','compulsion','obsession']]
new_test_df['compulsion_prediction'] = predicted_labels[:,0]
new_test_df['obsession_prediction'] = predicted_labels[:,1]
new_test_df

C:\Users\bkj\AppData\Local\Temp\ipykernel_22684\4254819293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df['compulsion_prediction'] = predicted_labels[:,0]
C:\Users\bkj\AppData\Local\Temp\ipykernel_22684\4254819293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_df['obsession_prediction'] = predicted_labels[:,1]


,post,compulsion,obsession,compulsion_prediction,obsession_prediction
629,hiya wanted bit advice really clomipramine yea...,0,1,0,1
1372,hello going little long sorry october 2016 ocd...,1,1,0,1
575,hi started post quite time always check see ha...,1,1,1,0
252,although obsession quite bad since comng uni r...,0,1,0,1
1064,annoyed feeling great lately managing obsessio...,1,1,1,0
...,...,...,...,...,...
894,dear friend making progress new medication reg...,1,1,1,0
315,dear friend severely struggling pure ocd many ...,0,1,0,1
1017,hi ocd make always revisit issue converted chr...,0,1,0,1
854,curiosity would say weirdest maybe paranoid th...,1,0,1,0


In [84]:
new_test_df.to_excel('new_test_df.xlsx')

In [129]:
sample

,post,prefLabel,Document_Keywords,yy,obsession_first_checker,compulsion_first_checker
0,"I grew up with obsessive thoughts, or more apt...","['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",grew obsess thought aptli intrus thought belie...,1.0,1.0
1,I have been thinking so much about Covid becau...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",think much covid part job well isnt right conc...,0.0,1.0
2,"I fed up, I send all day checking excessive ch...","['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",fed send day check excess check everi thing go...,0.0,1.0
3,Iâ€™ve noticed that Iâ€™ve been washing my han...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",ive notic ive wash hand late thing trigger nee...,1.0,0.0
4,This sounds so stupid\nAt the moment I want to...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",sound stupid moment want download window live ...,1.0,1.0
...,...,...,...,...,...,...
92,Saw an out of hours gp about everything and th...,[],"['hour', 'gp', 'waste', 'time', 'abuse', 'nhs'...",saw hour gp everyth said wast time abus nh can...,1.0,0.0
93,just woke up... even though i feel like a zomb...,['OCD'],"['wake', 'feel', 'zombie', 'wake', 'half', 'ha...",woke even though feel like zombi woke half han...,0.0,0.0
94,On friday I walked out of work so I could see ...,['Treatment'],"['walk', 'work', 'gp', 'minute', 'thing', 'tou...",friday walk work could see gp last minut thing...,0.0,0.0
95,All my worries are about saying or implying ba...,"['syncope', 'anger', 'hope', 'Thought']","['worry', 'term', 'untoward', 'early', 'friend...",worri say impli bad dad especi term say he don...,0.0,0.0


In [130]:
sample = clean_text(sample)
sample['post'] = sample['post'].apply(process_text)
sample['yy'] = sample['yy'].apply(process_text)
sample['clean_text'] = sample['clean_text'].apply(process_text)
sample.head()

,post,prefLabel,Document_Keywords,yy,obsession_first_checker,compulsion_first_checker,clean_text
0,grew obsessive thought aptly intrusive thought...,"['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",grew ob thought aptli intrus thought believ re...,1.0,1.0,grew ob thought aptli intrus thought believ re...
1,thinking much covid part job well right concer...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",think much covid part job well right concern e...,0.0,1.0,think much covid part job well right concern e...
2,fed send day checking excessive checking every...,"['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",fed send day check excess check everi thing go...,0.0,1.0,fed send day check excess check everi thing go...
3,noticed washing hand lately thing triggering n...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",notic wash hand late thing trigger need wash h...,1.0,0.0,notic wash hand late thing trigger need wash h...
4,sound stupid moment want download window live ...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",sound stupid moment want download window live ...,1.0,1.0,sound stupid moment want download window live ...


In [131]:
train_x = sample['post']
train_x

0     grew obsessive thought aptly intrusive thought...
1     thinking much covid part job well right concer...
2     fed send day checking excessive checking every...
3     noticed washing hand lately thing triggering n...
4     sound stupid moment want download window live ...
                            ...                        
92    saw hour gp everything said wasted time abusin...
93    woke even though feel like zombie woke half ha...
94    friday walked work could see gp last minute th...
95    worry saying implying bad dad especially term ...
96    hello everyone second therapy session day hvg ...
Name: post, Length: 97, dtype: object

In [132]:
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}

In [135]:
predictions = model.predict(train_encodings)
predicted_labels = (tf.sigmoid(predictions.logits) > 0.3).numpy() 
predicted_labels = predicted_labels.astype('int')
predicted_labels

4/4 [==============================] - 132s 30s/step


array([[1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0,

##### PREDICTION ON POSTS-MANUAL CHECK

In [109]:
manual_df = pd.read_csv('posts-manual-chacek.csv')
manual_df

,post,prefLabel,Document_Keywords,yy,obsession_first_checker,compulsion_first_checker
0,"I grew up with obsessive thoughts, or more apt...","['site', 'compulsion', 'obsession', 'rash', 'p...","['grow', 'obsessive_thought', 'aptly', 'intrus...",grew obsess thought aptli intrus thought belie...,1.0,1.0
1,I have been thinking so much about Covid becau...,"['singing', 'washing hands', 'anxiety', 'anxie...","['think', 'covid', 'job', 'concern', 'encourag...",think much covid part job well isnt right conc...,0.0,1.0
2,"I fed up, I send all day checking excessive ch...","['OCD', 'checking', 'Thought']","['send', 'day', 'check', 'excessive', 'checkin...",fed send day check excess check everi thing go...,0.0,1.0
3,I’ve noticed that I’ve been washing my hands m...,"['washing hands', 'hope', 'depression', 'OCD']","['notice', 'washing_hand', 'thing', 'trigger',...",ive notic ive wash hand late thing trigger nee...,1.0,0.0
4,This sounds so stupid\nAt the moment I want to...,['Thought'],"['sounds_stupid', 'moment', 'download', 'windo...",sound stupid moment want download window live ...,1.0,1.0
...,...,...,...,...,...,...
23451,Does anyone else feel like when trying to stop...,"['talking', 'Thought']","['feel', 'ruminate', 'respond', 'take', 'effor...",anyon els feel like tri stop rumin tri respond...,NaN,NaN
23452,So for the past year or two I have had repetat...,['Thought'],"['year', 'repetative', 'thought', 'brain_start...",past year two repet thought mainli brain start...,NaN,NaN
23453,Hi Guys I havent been around for a while went ...,['hope'],"['guy', 'med', 'high', 'doze', 'move', 'holida...",hi guy havent around went back old med highest...,NaN,NaN
23454,My little baby is due in 2 weeks exactly (hopi...,"['Thought', 'joy', 'hope', 'process']","['baby', 'week', 'hope', 'perfect', 'time', 's...",littl babi due week exactli hope go perfect ti...,NaN,NaN


In [110]:
manual_df = manual_df[['post','obsession_first_checker','compulsion_first_checker','yy']]
manual_df

,post,obsession_first_checker,compulsion_first_checker,yy
0,"I grew up with obsessive thoughts, or more apt...",1.0,1.0,grew obsess thought aptli intrus thought belie...
1,I have been thinking so much about Covid becau...,0.0,1.0,think much covid part job well isnt right conc...
2,"I fed up, I send all day checking excessive ch...",0.0,1.0,fed send day check excess check everi thing go...
3,I’ve noticed that I’ve been washing my hands m...,1.0,0.0,ive notic ive wash hand late thing trigger nee...
4,This sounds so stupid\nAt the moment I want to...,1.0,1.0,sound stupid moment want download window live ...
...,...,...,...,...
23451,Does anyone else feel like when trying to stop...,NaN,NaN,anyon els feel like tri stop rumin tri respond...
23452,So for the past year or two I have had repetat...,NaN,NaN,past year two repet thought mainli brain start...
23453,Hi Guys I havent been around for a while went ...,NaN,NaN,hi guy havent around went back old med highest...
23454,My little baby is due in 2 weeks exactly (hopi...,NaN,NaN,littl babi due week exactli hope go perfect ti...


In [111]:
manual_df['post'] = manual_df['post'].apply(process_text)
manual_df.head()

C:\Users\bkj\AppData\Local\Temp\ipykernel_22684\3355615258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual_df['post'] = manual_df['post'].apply(process_text)


,post,obsession_first_checker,compulsion_first_checker,yy
0,grew obsessive thought aptly intrusive thought...,1.0,1.0,grew obsess thought aptli intrus thought belie...
1,thinking much covid part job well right concer...,0.0,1.0,think much covid part job well isnt right conc...
2,fed send day checking excessive checking every...,0.0,1.0,fed send day check excess check everi thing go...
3,noticed washing hand lately thing triggering n...,1.0,0.0,ive notic ive wash hand late thing trigger nee...
4,sound stupid moment want download window live ...,1.0,1.0,sound stupid moment want download window live ...


In [114]:
manual_train_x = manual_df['post']

In [116]:
train_encodings = tokenizer(manual_train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}

In [117]:
y_pred = model.predict(train_encodings)
y_pred


733/733 [==============================] - 17946s 24s/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 2.300557  , -0.40719262],
       [-1.8444637 ,  2.7699006 ],
       [-2.0910683 ,  2.7794967 ],
       ...,
       [-2.0358596 ,  2.824232  ],
       [-1.8154553 ,  2.7890458 ],
       [-1.8086473 ,  2.75507   ]], dtype=float32), hidden_states=None, attentions=None)

In [118]:
y_pred = (tf.sigmoid(y_pred.logits) > 0.5).numpy()
y_pred

array([[ True, False],
       [False,  True],
       [False,  True],
       ...,
       [False,  True],
       [False,  True],
       [False,  True]])

In [119]:
y_pred = y_pred.astype('int')
y_pred

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]])

In [120]:
manual_df['compulsion_pred'] = y_pred[:,0]
manual_df['obsession_pred'] = y_pred[:,1]
manual_df

C:\Users\bkj\AppData\Local\Temp\ipykernel_22684\1666728390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual_df['compulsion_pred'] = y_pred[:,0]


,post,obsession_first_checker,compulsion_first_checker,yy,compulsion_pred,obsession_pred
0,grew obsessive thought aptly intrusive thought...,1.0,1.0,grew obsess thought aptli intrus thought belie...,1,0
1,thinking much covid part job well right concer...,0.0,1.0,think much covid part job well isnt right conc...,0,1
2,fed send day checking excessive checking every...,0.0,1.0,fed send day check excess check everi thing go...,0,1
3,noticed washing hand lately thing triggering n...,1.0,0.0,ive notic ive wash hand late thing trigger nee...,0,1
4,sound stupid moment want download window live ...,1.0,1.0,sound stupid moment want download window live ...,0,1
...,...,...,...,...,...,...
23451,anyone else feel like trying stop ruminating t...,NaN,NaN,anyon els feel like tri stop rumin tri respond...,0,1
23452,past year two repetative thought mainly brain ...,NaN,NaN,past year two repet thought mainli brain start...,0,1
23453,hi guy around went back old med highest doze s...,NaN,NaN,hi guy havent around went back old med highest...,0,1
23454,little baby due 2 week exactly hoping go perfe...,NaN,NaN,littl babi due week exactli hope go perfect ti...,0,1


In [121]:
manual_df.to_excel('post-manual_prediction_without-ontology.xlsx')

In [ ]:
process_df

In [122]:
test_df

,i,post,prefLabel,compulsion,obsession,yy,clean_text
629,630,hiya wanted bit advice really clomipramine yea...,"['OCD', 'Thought', 'Intrusive thoughts']",0,1,intrusive thought aggressive intrusive thought...,intrusive thought aggressive intrusive thought...
1372,1373,hello going little long sorry october 2016 ocd...,"['symptom', 'drop', 'hope', 'site', 'compulsio...",1,1,compulsion intrusive thought aggressive intrus...,compulsion intrusive thought aggressive intrus...
575,576,hi started post quite time always check see ha...,"['Medication', 'obsession', 'checking', 'compu...",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
252,253,although obsession quite bad since comng uni r...,"['napping', 'obsession', 'OCD']",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1064,1065,annoyed feeling great lately managing obsessio...,"['obsession', 'Thought', 'compulsion', 'OCD']",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
...,...,...,...,...,...,...,...
894,895,dear friend making progress new medication reg...,"['Medication', 'discharge', 'obsession', 'comp...",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
315,316,dear friend severely struggling pure ocd many ...,"['Suicidal', 'obsession', 'Intrusive thoughts'...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1017,1018,hi ocd make always revisit issue converted chr...,"['anxiety', 'anxiety', 'Intrusive thoughts', '...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
854,855,curiosity would say weirdest maybe paranoid th...,"['OCD', 'Thought', 'compulsion']",1,0,compulsion thought morbid thought mental image...,compulsion thought morbid thought mental image...


#### MODEL TRAINING WITH ONTOLOGY AND POST

In [5]:
df_ontology = pd.read_excel('posts_with_label_and_pref_and_expand1-2.xlsx')
df_ontology

,post,prefLabel,compulsion,obsession,yy
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination..."
...,...,...,...,...,...
1556,"Hi all,\nI think I know the answers before I e...","['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
1557,"Hi folks,\nI'm really struggling at the mo. I'...","['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination..."
1558,Well hello everyone\nAs you can see one of my ...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination..."
1559,Well hello!\n\nIve been struggling with ocd in...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination..."


In [6]:
#apply the process of cleaning for the train and test data
df_ontology = clean_text(df_ontology)
df_ontology['post'] = df_ontology['post'].apply(process_text)
df_ontology['yy'] = df_ontology['yy'].apply(process_text)
df_ontology['clean_text'] = df_ontology['clean_text'].apply(process_text)
df_ontology.head()


,post,prefLabel,compulsion,obsession,yy,clean_text
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,compulsion intrusive thought aggressive intrus...,compulsion intrusive thought aggressive intrus...
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,obsession aggressive obsession contamination o...,obsession aggressive obsession contamination o...


In [7]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(df_ontology, test_size=test_split, stratify=df_ontology[['compulsion', 'obsession']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['compulsion', 'obsession']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 1248
Number of rows in validation set: 157
Number of rows in test set: 156


In [8]:
train_x,train_ontology,train_y = train_df['post'],train_df['yy'],train_df[['compulsion','obsession']]
test_x,test_ontology,test_y = test_df['post'],test_df['yy'],test_df[['compulsion','obsession']]
val_x,val_ontology,val_y = val_df['post'],val_df['yy'],val_df[['compulsion','obsession']]

In [9]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, classification_report

# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Assuming you have separate text and ontology data
text_data = train_x.tolist()  # Replace with your actual text data
ontology_data = train_ontology.tolist()  # Replace with your actual ontology data

# Tokenize and encode the text data
text_encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors='tf')
text_outputs = model(**text_encodings)

# Tokenize and encode the ontology data
ontology_encodings = tokenizer(ontology_data, truncation=True, padding=True, max_length=128, return_tensors='tf')
ontology_outputs = model(**ontology_encodings)

# Combine representations (for example, concatenate or sum)
combined_representation = tf.concat([text_outputs.last_hidden_state, ontology_outputs.last_hidden_state], axis=1)

# Add a dense layer for classification
classifier = tf.keras.layers.Dense(2, activation='sigmoid')
output = classifier(combined_representation)

# Define the model
combined_model = tf.keras.Model(inputs=model.input, outputs=output)

# Compile the model
combined_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
history = combined_model.fit(
    text_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=3,
    batch_size=32
)

# Evaluate the model
predictions = combined_model.predict(test_encodings)
predicted_labels = (tf.sigmoid(predictions) > 0.5).numpy()
accuracy = accuracy_score(test_labels, predicted_labels)
confusion_matrix = multilabel_confusion_matrix(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", confusion_matrix)
print("Classification Report:\n", classification_rep)



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
